In [1]:
# Amend to set wd to STB_social_media_analytics.
%cd /home/zyf0717/git_environment/STB_social_media_analytics/

/home/zyf0717/git_environment/STB_social_media_analytics


In [2]:
import pandas as pd
import mysql.connector
import yaml
import utils
from selenium import webdriver
from datetime import datetime
from time import sleep

In [14]:
with open('config_file.yml') as file:
    configs = yaml.load(file, Loader=yaml.FullLoader)

cnx = mysql.connector.connect(host=configs['host'],
                              database=configs['database'],
                              user=configs['user'],
                              password=configs['password'])

driver = webdriver.Chrome('./chromedriver')

# Primary key is 'POI_index'.
TripA_POI_cols = ['POI_index',
                  'total_reviews',
                  'ranking',
                  'rating_1_proportion',
                  'rating_2_proportion',
                  'rating_3_proportion',
                  'rating_4_proportion',
                  'rating_5_proportion',
                  'about',
                  'price',
                  'address',
                  'latitude',
                  'longitude',
                  'attributes_crawled_time'
                 ]

# Primary key is 'review_index', autoincrement in DB.
TripA_reviews_cols = ['POI_indes',
                      'reviewer_name',
                      'raw_home_location',
                      'cleaned_home_location',
                      'number_of_contributions',
                      'helpful_votes',
                      'review_date',
                      'review_title',
                      'review_body',
                      'date_of_experience',
                      'trip_type',
                      'review_cralwed_time'
                     ]

poi_df = pd.DataFrame(columns=TripA_POI_cols)

### MVP ###
url_1 = 'https://www.tripadvisor.com.sg/Attraction_Review-g294265-d2149128-Reviews-Gardens_by_the_Bay-Singapore.html'
url_2 = 'https://www.tripadvisor.com.sg/Attraction_Review-g294265-d1837767-Reviews-Marina_Bay_Sands_Skypark-Singapore.html'
driver.get(url_1)

# Attributes.

# total_reviews_text = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div/div[3]/div/div[1]/div/div[1]/a/span/text()').get()
# total_reviews = int(total_reviews_text[:total_reviews_text.find(" ")].replace(",", ""))
# ranking_text = sel.xpath('/html/body/div[2]/div[1]/div/div[3]/div/div[1]/div/div[2]/div/span/b/span/text()').get()
# ranking = int(ranking_text[1:])
# rating_text = sel.xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div/div/div[1]/div/div/div[2]/span/text()').get()
# rating_1_proportion_text = sel.xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div/div/div[1]/div/div/div[3]/div/ul/li[1]/span[3]/text()').get()
# rating_2_proportion_text = sel.xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div/div/div[1]/div/div/div[3]/div/ul/li[2]/span[3]/text()').get()
# rating_3_proportion_text = sel.xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div/div/div[1]/div/div/div[3]/div/ul/li[3]/span[3]/text()').get()
# rating_4_proportion_text = sel.xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div/div/div[1]/div/div/div[3]/div/ul/li[4]/span[3]/text()').get()
# rating_5_proportion_text = sel.xpath('/html/body/div[2]/div[2]/div[2]/div[3]/div/div/div[1]/div/div/div[3]/div/ul/li[5]/span[3]/text()').get()
# price_text = sel.xpath('/html/body/div[2]/div[1]/div/div[5]/div/div[2]/div/div/div/div[1]/div[2]/div[1]/span/div/div/div/div[2]/div/span/span/text()').get()


total_reviews_text = driver.find_element_by_xpath('//span[@class="reviewCount"]').text
# total_reviews = int(total_reviews_text[:total_reviews_text.find(" ")].replace(",", ""))
ranking_text = driver.find_element_by_xpath('//span[@class="header_popularity popIndexValidation "]').text
rating_text = driver.find_element_by_xpath('//span[@class="overallRating"]').text
rating_proportion_elements = driver.find_elements_by_xpath('//span[@class="row_count row_cell"]')
address_text_1 = driver.find_element_by_xpath('//span[@class="street-address"]').text
address_text_2 = driver.find_element_by_xpath('//span[@class="extended-address"]').text
address_text_3 = driver.find_element_by_xpath('//span[@class="locality"]').text
address_text_4 = driver.find_element_by_xpath('//span[@class="country-name"]').text
try:
    about_more_button = driver.find_elements_by_xpath('//span[@class="attractions-attraction-detail-about-card-Description__readMore--2pd33"]')[0]
    about_more_button.click()
    sleep(0.5)
    about_text = driver.find_element_by_xpath('//div[@class="attractions-attraction-detail-about-card-Description__modalText--1oJCY"]').text
    about_more_close_button = driver.find_element_by_xpath('//div[@class="_2EFRp_bb"]')
    about_more_close_button.click()
    sleep(0.5)
    
except:
    about_text = driver.find_element_by_xpath('//div[@class="attractions-attraction-detail-about-card-AttractionDetailAboutCard__section--1_Efg"]').text

attributes_crawled_time = datetime.now()

print(total_reviews_text)
print(ranking_text)
print(rating_text)
for element in rating_proportion_elements:
    print(element.text)
print(about_text)
print(address_text_1)
print(address_text_2)
print(address_text_3)
print(address_text_4)
print(attributes_crawled_time)
print('\n### END OF ATTRIBUTES ###\n')

# Reviews
read_more_button = driver.find_element_by_xpath('//span[@class="location-review-review-list-parts-ExpandableReview__cta--2mR2g"]')
read_more_button.click()
sleep(0.5)

reviewer_url_elements = driver.find_elements_by_xpath('//a[@class="ui_header_link social-member-event-MemberEventOnObjectBlock__member--35-jC"]')
# helpful_votes_elements = driver.find_elements_by_xpath('//[@]')
review_rating_elements = driver.find_elements_by_xpath('//div[@class="location-review-review-list-parts-RatingLine__bubbles--GcJvM"]')
review_date_elements = driver.find_elements_by_xpath('//div[@class="social-member-event-MemberEventOnObjectBlock__event_type--3njyv"]')
review_title_elements = driver.find_elements_by_xpath('//a[@class="location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT"]')
review_body_elements = driver.find_elements_by_xpath('//q[@class="location-review-review-list-parts-ExpandableReview__reviewText--gOmRC"]')
date_of_experience_elements = driver.find_elements_by_xpath('//span[@class="location-review-review-list-parts-EventDate__event_date--1epHa"]')
# trip_type_elements = driver.find_elements_by_xpath('//[@]')

reviewer_name_elements = reviewer_url_elements
# raw_home_location_elements = driver.find_elements_by_xpath('//span[@class=class="default social-member-common-MemberHometown__hometown--3kM9S small"]')
# number_of_contribution_elements = driver.find_elements_by_xpath('//[@]')

review_crawled_time = datetime.now()

for i in range(len(review_body_elements)):
    print(reviewer_url_elements[i].get_attribute('href'))
    print(reviewer_name_elements[i].text)
    print(review_date_elements[i].text)
    print(review_rating_elements[i].get_attribute('class'))
    print(review_title_elements[i].text)
    print(review_body_elements[i].text)
    print(date_of_experience_elements[i].text + "\n")
    
    
next_button = driver.find_element_by_xpath('//a[@class="ui_button nav next primary "]')
next_button.click()
sleep(0.5)

58,404 Reviews
#3 of 975 things to do in Singapore
4.5
71%
23%
4%
1%
1%
An integral part of Singapore's "City in a Garden" vision, Gardens by the Bay spans a total of 101 hectares of prime land at the heart of Singapore's new downtown - Marina Bay. Comprising three waterfront gardens - Bay South, Bay East and Bay Central - Gardens by the Bay will be a showcase of horticulture and garden artistry that will bring the world of plants to Singapore and present Singapore to the World.
18 Marina Gardens Drive
Bayfront Plaza
Singapore 018953,
Singapore
2020-01-10 18:01:35.979302

### END OF ATTRIBUTES ###

https://www.tripadvisor.com.sg/Profile/crystaldreams
crystaldreams
crystaldreams wrote a review Yesterday
location-review-review-list-parts-RatingLine__bubbles--GcJvM
Gardens by the Bay at Christmas
Pretty crowded and over the top Christmas decorations in addition to the normal gardens. I think I would prefer it during a regular time instead of Christmas.
Date of experience: December 2019

h